#  Setup stuff: Connectivity

In [ ]:

#  Setting display width 

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Setup stuff: Debugging 

In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:

l_result = my_graph.query("""

   MATCH ( n ) - [ r ] -> ( m )
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


In [ ]:

l_result = my_graph.query("""

   MATCH ( n )
   RETURN n
   
   """, contextualize=True)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Before running sssp()
#
# +----+-----------------+-------------+-----------+------------------+--------+----------+
# |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
# |----+-----------------+-------------+-----------+------------------+--------+----------|
# |  0 |               0 | ['Airport'] | Airport   | Denver           | DEN    | node     |
# |  1 |               1 | ['Airport'] | Airport   | San Jose         | SJC    | node     |
# |  2 |               2 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
# |  3 |               3 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    | node     |
# +----+-----------------+-------------+-----------+------------------+--------+----------+

#  After running sssp()
#
# +----+-----------------+-------------+-----------+------------------+--------+--------------------+----------+
# |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   |   n.sssp_dist_0001 | n.type   |
# |----+-----------------+-------------+-----------+------------------+--------+--------------------+----------|
# |  0 |               0 | ['Airport'] | Airport   | Denver           | DEN    |                952 | node     |
# |  1 |               1 | ['Airport'] | Airport   | San Jose         | SJC    |               1900 | node     |
# |  2 |               2 | ['Airport'] | Airport   | Milwaukee        | MKE    |                  0 | node     |
# |  3 |               3 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    |                 66 | node     |
# +----+-----------------+-------------+-----------+------------------+--------+--------------------+----------+


In [ ]:

l_result = my_graph.query("""

   MATCH ( n ) - [ r ] -> ( m )
   RETURN r

   
   """, contextualize=True)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Before running sssp()
#
# +----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------+
# |    |   r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   r.start.id | r.start.labels   | r.type   |
# |----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------|
# |  0 |          0 | ['Airport']    |               0 | FLIES_TO  |          948 |            1 | FLIES_TO |            1 | ['Airport']      | edge     |
# |  1 |          0 | ['Airport']    |               1 | FLIES_TO  |          886 |            1 | FLIES_TO |            3 | ['Airport']      | edge     |
# |  2 |          1 | ['Airport']    |               2 | FLIES_TO  |          948 |            1 | FLIES_TO |            0 | ['Airport']      | edge     |
# |  3 |          2 | ['Airport']    |               3 | FLIES_TO  |           66 |            1 | FLIES_TO |            3 | ['Airport']      | edge     |
# |  4 |          3 | ['Airport']    |               4 | FLIES_TO  |          886 |            1 | FLIES_TO |            0 | ['Airport']      | edge     |
# |  5 |          3 | ['Airport']    |               6 | FLIES_TO  |           66 |            1 | FLIES_TO |            2 | ['Airport']      | edge     |
# |  6 |          3 | ['Airport']    |               5 | FLIES_TO  |         1829 |            1 | FLIES_TO |            1 | ['Airport']      | edge     |
# +----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------+

#  After running sssp()
#
# +----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------+
# |    |   r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   r.start.id | r.start.labels   | r.type   |
# |----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------|
# |  0 |          0 | ['Airport']    |               0 | FLIES_TO  |          948 |            1 | FLIES_TO |            1 | ['Airport']      | edge     |
# |  1 |          0 | ['Airport']    |               1 | FLIES_TO  |          886 |            1 | FLIES_TO |            3 | ['Airport']      | edge     |
# |  2 |          1 | ['Airport']    |               2 | FLIES_TO  |          948 |            1 | FLIES_TO |            0 | ['Airport']      | edge     |
# |  3 |          2 | ['Airport']    |               3 | FLIES_TO  |           66 |            1 | FLIES_TO |            3 | ['Airport']      | edge     |
# |  4 |          3 | ['Airport']    |               4 | FLIES_TO  |          886 |            1 | FLIES_TO |            0 | ['Airport']      | edge     |
# |  5 |          3 | ['Airport']    |               6 | FLIES_TO  |           66 |            1 | FLIES_TO |            2 | ['Airport']      | edge     |
# |  6 |          3 | ['Airport']    |               5 | FLIES_TO  |         1829 |            1 | FLIES_TO |            1 | ['Airport']      | edge     |
# +----+------------+----------------+-----------------+-----------+--------------+--------------+----------+--------------+------------------+----------+


#  If the RETURN is,
#     RETURN r.type, r.TYPE, r.label, r.start.labels, r.end.labels, r.internal_id, r.start.id, r.end.id, r.DISTANCE, r.NUM_HOPS
#
# +----+--------------+--------------+----------+------------+----------------+-----------------+-----------+--------------+------------------+----------+
# |    |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   | r.end.id   | r.end.labels   | r.internal_id   | r.label   | r.start.id   | r.start.labels   | r.type   |
# |----+--------------+--------------+----------+------------+----------------+-----------------+-----------+--------------+------------------+----------|
# |  0 |           66 |            1 | FLIES_TO |            |                |                 |           |              |                  |          |
# |  1 |          948 |            1 | FLIES_TO |            |                |                 |           |              |                  |          |
# |  2 |         1829 |            1 | FLIES_TO |            |                |                 |           |              |                  |          |
# |  3 |          886 |            1 | FLIES_TO |            |                |                 |           |              |                  |          |
# |  4 |           66 |            1 | FLIES_TO |            |                |                 |           |              |                  |          |
# |  5 |          948 |            1 | FLIES_TO |            |                |                 |           |              |                  |          |
# |  6 |          886 |            1 | FLIES_TO |            |                |                 |           |              |                  |          |
# +----+--------------+--------------+----------+------------+----------------+-----------------+-----------+--------------+------------------+----------+


#  Actual analytics ..

In [ ]:

from katana import remote
from katana.remote import analytics


l_cntr = 0

print("--")


In [18]:

#  Running analytics.sssp() ..


l_source_nodeid       = "2"                        #  id of node to start on (this is MKE)
l_weight_property     = "DISTANCE"                 #  Edge property
l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


          0/? [?op/s]

--


None

In [19]:

l_result = my_graph.query("""

   MATCH ( n )
   RETURN n
   
   """, contextualize=True)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Before running sssp()
#
# +----+-----------------+-------------+-----------+------------------+--------+----------+
# |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
# |----+-----------------+-------------+-----------+------------------+--------+----------|
# |  0 |               0 | ['Airport'] | Airport   | Denver           | DEN    | node     |
# |  1 |               1 | ['Airport'] | Airport   | San Jose         | SJC    | node     |
# |  2 |               2 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
# |  3 |               3 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    | node     |
# +----+-----------------+-------------+-----------+------------------+--------+----------+

#  After running sssp()
#
# +----+-----------------+-------------+-----------+------------------+--------+--------------------+----------+
# |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   |   n.sssp_dist_0001 | n.type   |
# |----+-----------------+-------------+-----------+------------------+--------+--------------------+----------|
# |  0 |               0 | ['Airport'] | Airport   | Denver           | DEN    |                952 | node     |
# |  1 |               1 | ['Airport'] | Airport   | San Jose         | SJC    |               1900 | node     |
# |  2 |               2 | ['Airport'] | Airport   | Milwaukee        | MKE    |                  0 | node     |
# |  3 |               3 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    |                 66 | node     |
# +----+-----------------+-------------+-----------+------------------+--------+--------------------+----------+


          0/? [?op/s]

          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+--------+--------------------+--------------------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   |   n.sssp_dist_0001 |   n.sssp_dist_0002 | n.type   |
|----+-----------------+-------------+-----------+------------------+--------+--------------------+--------------------+----------|
|  0 |               3 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    |                 66 |                 66 | node     |
|  1 |               1 | ['Airport'] | Airport   | San Jose         | SJC    |               1900 |               1900 | node     |
|  2 |               0 | ['Airport'] | Airport   | Denver           | DEN    |                952 |                952 | node     |
|  3 |               2 | ['Airport'] | Airport   | Milwaukee        | MKE    |                  0 |                  0 | node     |
+----+-----------------+-------------+-----------+------------------+-------